In [1]:
import pandas as pd
import re
import sys
from PyPDF2 import PdfFileReader
import docx2txt
import openpyxl
import gspread
import df2gspread as d2g

In [2]:
#Extract pdf text
file = open('/Users/hector/Desktop/pau/cv.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " " #empty variable for the extracted text
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

                 COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 02/03/2022                                           RAN ON FEBRUARY 03, 2022 09:32am PAGE: 1        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY        00-02-16392-CV     02/04/2000 (TITLE IV-D OAG USE ONLY                                                   ATTORNEY GENERAL              MELINDA S. LAUCK          CHAVARRIA, JOSE M.                                         CHAVARRIA, AZUCENA                                                                                                 -------------------------------------------------------------------------------------------------------------        1-01-33838-MCV     01/11/2017 MOTOR VEHICLE                                                              CANO, ANA                     ROLANDO SAL

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#regex - finds and extracts cause_numbers, file_date, cause_action(24 character), docket_date, docket-type, plaintiff (21 character)
finds_cause_numbers = re.findall(r'(\d{2}-\d{2}-\d{5}-\w*|\d*-\d*-\d*-\w*|\d*-\w*)\s*(\d{2}/\d{2}/\d{4})\s*(\D.{23})\s*(\d{2}/\d{2}/\d{4}|[ ]{0,1})(\D.[M$][O$][T$][I$].{,12}|\D.[W$][O$][N$]..{,12}|\D.[T$][R$][I$]..{,12}|\D.[J$][U$][R$]..{,12}|\D.[S$][T$][A$]..{,12}|\D.[H$][E$][A$]..{,12}|\D.[P$][R$][E$]..{,12}|\D.[E$][N$][T$]..{,12}|\D.[P$][E$][T$]..{,12}|\D.[F$][I$][N$]..{,12}|\D.[C$][O$][M$]..{,12}|\D.[P$][L$][E$]..{,12}|\D.[N$][O$][T$]..{,12}|[ ]{0,1})\s(\D.{,20})', content)#pattern for cause number
finds_cause_numbers


[('00-02-16392-CV',
  '02/04/2000',
  '(TITLE IV-D OAG USE ONLY',
  '',
  '',
  'ATTORNEY GENERAL     '),
 ('1-01-33838-MCV',
  '01/11/2017',
  'MOTOR VEHICLE           ',
  '',
  '',
  'CANO, ANA            '),
 ('1-03-34076-MCV',
  '03/09/2017',
  'DIVORCE - CHILDREN      ',
  '',
  '',
  'MORALES, SYLVIA DANIE'),
 ('01-06-17430-CV',
  '11/09/2017',
  'SHOW CAUSE MOTION       ',
  '08/29/2001',
  '  PETITION TO ESTA',
  '                    A'),
 ('1-07-34542-MCV',
  '07/17/2017',
  'DIVORCE - CHILDREN      ',
  '',
  '',
  'RODRIGUEZ AVALOS, ALE'),
 ('01-12-17744-MCV',
  '12/06/2001',
  'OTHER CIVIL CASES       ',
  '04/12/2002',
  '  TRIAL ON THE MER',
  '                    S'),
 ('02-06-18350-MCV',
  '06/05/2002',
  '(TITLE IV-D OAG USE ONLY',
  '',
  '',
  'ATTORNEY GENERAL     '),
 ('02-07-18488-MCV',
  '07/17/2002',
  '(TITLE IV-D OAG USE ONLY',
  '',
  '',
  'ATTORNEY GENERAL     '),
 ('02-08-18574-MCV',
  '08/21/2002',
  'INJURY OR DAMAGE INVOLVI',
  '10/16/2002',
  '  PRE-T

In [4]:
pending_cause_number_df = pd.DataFrame(finds_cause_numbers, columns = ['cause_number', 'file_date', 'cause_of_action', 'docket_date', 'docket_type', "plaintiff"])#converts list into df
pending_cause_number_df.head()


,cause_number,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,00-02-16392-CV,02/04/2000,(TITLE IV-D OAG USE ONLY,,,ATTORNEY GENERAL
1,1-01-33838-MCV,01/11/2017,MOTOR VEHICLE,,,"CANO, ANA"
2,1-03-34076-MCV,03/09/2017,DIVORCE - CHILDREN,,,"MORALES, SYLVIA DANIE"
3,01-06-17430-CV,11/09/2017,SHOW CAUSE MOTION,08/29/2001,PETITION TO ESTA,A
4,1-07-34542-MCV,07/17/2017,DIVORCE - CHILDREN,,,"RODRIGUEZ AVALOS, ALE"


In [ ]:
#df.to_csv('/Users/hector/Desktop/pau/cause.csv')#writes to csv

In [ ]:
#I used the merge function to add the previous pending cases with the new one that contained file_date, cause_of_action, and so forth.
#It used once to gather both data sets. Now that the dataset is one we will go back to using the .append function.
#df = civil_pending_notes.merge(pending_cause_number_df, how = 'outer', on = 'cause_number')
#df


In [5]:
#opens the google sheet of pending case notes
    #sets the json to service account path
json_path = gspread.service_account(filename = '/Users/hector/codeup-data-science/293pending_cases/pending_cases.json')
    #opens the google sheet by key found in the address
opens_civil_pending_gs = json_path.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')
    #pulls the data from the google worksheet (civil_pending_notes tab)
civil_pending_notes_tab = opens_civil_pending_gs.get_worksheet(0)
    #puts the data from the google sheet and puts it into a dataframe
civil_pending_notes = pd.DataFrame(civil_pending_notes_tab.get_all_records())
civil_pending_notes.shape

(2983, 11)

In [9]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df), ignore_index=True)
df.head()


,cause_number,notes,disposed,dispose_date,on_track,notes_lupita,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,1,(Look for this case under Plaintiff name G and...,,,,,,,,,
1,2,Case does not have any information. Please dis...,,,,,,,,,
2,6972,DISPOSED,TRUE,3/30/2011,,,,,,,
3,7038,DISPOSED,TRUE,2/11/1999,,,,,,,
4,8672,DISPOSED,TRUE,11/14/2005,,,,,,,


In [ ]:
#df.to_csv('/Users/hector/Desktop/pau/cause.csv')#writes to csv

In [8]:
    #drops the duplicated cause numbers and reindexes the dataframe
    #resets the index and drops the output index
    #fills in the na with an empty space to avoid error
df = df.drop_duplicates('cause_number').reset_index(drop=True).fillna(' ')
df

,cause_number,notes,disposed,dispose_date,on_track,notes_lupita,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,1,(Look for this case under Plaintiff name G and...,,,,,,,,,
1,2,Case does not have any information. Please dis...,,,,,,,,,
2,6972,DISPOSED,TRUE,3/30/2011,,,,,,,
3,7038,DISPOSED,TRUE,2/11/1999,,,,,,,
4,8672,DISPOSED,TRUE,11/14/2005,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
2979,22-02-40784-MCV,,,,,,,,,,
2980,22-02-40786-MCV,,,,,,,,,,
2981,22-02-40788-MCV,,,,,,,,,,
2982,22-02-40790-MCV,,,,,,,,,,


In [ ]:
(df['notes'].values == ' ').sum()




In [ ]:
df.to_csv('/Users/hector/Desktop/pau/causenew.csv')#writes to csv

In [ ]:
total = df.cause_number.count()
total

In [ ]:
disposed = (df['disposed']).value_counts()['TRUE']
disposed

In [ ]:
pending = total - disposed
pending

In [ ]:
#civil_pending_notes_tab.clear()

In [ ]:
#civil_pending_notes_tab.update([df.columns.values.tolist()] + df.values.tolist())

# SCRATCH code

In [ ]:
pending_cause_number_df

In [ ]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df, columns=['cause_number']), ignore_index=True)
df

In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates('cause_number').fillna(' ').reset_index(drop=True)
df

In [ ]:
# add a sheet with 20 rows and 2 columns
opens_civil_pending_gs.add_worksheet(rows=2000,cols=10,title='new')

# get the instance of the second sheet
civil_pending_notes = opens_civil_pending_gs.get_worksheet(1)

In [ ]:
    #updates the google sheet with the new list of pending cases
civil_pending_notes.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
civil_pending_notes

In [ ]:
rangeAll = '{0}!A1:Z'.format( sheetName )
body = {}
resultClear = service.spreadsheets( ).batchUpdate().values( ).clear( spreadsheetId=spreadsheet_id, range=rangeAll,
                                                       body=body ).execute( )

In [ ]:
gc = gspread.service_account(filename = '/Users/hector/Desktop/pau/pending_cases.json')
sht1 = gc.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')


In [ ]:
worksheet = sht1.get_worksheet(0)

In [ ]:

civil_pending_notes = pd.DataFrame(worksheet.get_all_records())


In [ ]:
#civil_pending_notes = pd.read_excel('civil_pending.xlsx')
civil_pending_notes


In [ ]:
df = df.fillna(' ')
df

In [ ]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_cause_numbers.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('yesterdays_pending.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts cause numbers from the previous day's pending report
y_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
yesterday_df = pd.DataFrame(y_causenum, columns = ['cause_number'])#puts it into a df
yesterday_df.shape

In [ ]:
#adds both lists together in order to search for dups later
df = todays_df.append(pd.DataFrame(y_causenum, columns=['cause_number']), ignore_index=True)
df.shape #shape of total rows


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates()
df.shape

In [ ]:
df

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_causes.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#reads in the most recent excel file that has notes ie 'latest_df'
latest_df = pd.read_excel('pending_causes.xlsx', index_col=0)#index_col = 0 so that it does not add in another indexed col
#this will read in a google sheet that has the notes in it


In [ ]:
latest_df

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('7_pen.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts new day report cause numbers to be compared to latest report
t_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
todays_df = pd.DataFrame(t_causenum, columns = ['cause_number'])

In [ ]:
todays_df

In [ ]:
#takes the new day report and adds it to the latest report for comparison
df = latest_df.append(pd.DataFrame(todays_df, columns=['cause_number']), ignore_index=True) #adds both lists together in order to search for dups later
df


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates() 
df
###Needs to be reindexes!!

In [ ]:
#prints back to excel
#update this file
df.to_excel("_pending_causes.xlsx", sheet_name='pending_cause_numbers')  

#New version of this will go back to the google sheet

In [ ]:
#duplicated=df[df.duplicated()] #finds all duplicates
#duplicated.shape #count of rows and columns of the duplicated cause numbers

In [ ]:
#shows ONLY the duplicates
#duplicated.sort_values('cause_number')